# 第二章 模型，提示和输出解释器

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、直接使用OpenAI](#二、直接使用OpenAI)
     - [2.1 计算1+1](#2.1-计算1+1)
     - [2.2 用美式英语表达海盗邮件](#2.2-用美式英语表达海盗邮件)
     - [2.3 中文版](#2.3-中文版)
 - [三、通过LangChain使用OpenAI](#三、通过LangChain使用OpenAI)
     - [3.1 模型](#3.1-模型)
     - [3.2 提示模板](#3.2-提示模板)
         - [3.2.1 使用LangChain提示模版](#3.2.1-使用LangChain提示模版)
         - [3.2.2 为什么需要提示模版](#3.2.2-为什么需要提示模版)
     - [3.3 输出解析器](#3.3-输出解析器)
         - [3.3.1 如果没有输出解析器](#3.3.1-如果没有输出解析器)
         - [3.3.2 LangChain输出解析器](#3.3.3-LangChain输出解析器)
 - [四、补充材料](#四、补充材料)
     - [4.1 链式思考推理(ReAct)](#4.1-链式思考推理(ReAct))


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [1]:
import sys
sys.path.append("../")

from src.utils.chat_robot import OpenAiChat

## 二、直接使用OpenAI

我们先从直接调用OpenAI的API开始。

`get_completion`函数是基于`openai`的封装函数，对于给定提示（prompt）输出相应的回答。其包含两个参数
   
   - `prompt` 必需输入参数。 你给模型的**提示，可以是一个问题，可以是你需要模型帮助你做的事**（改变文本写作风格，翻译，回复消息等等）。
   - `model` 非必需输入参数。默认使用gpt-3.5-turbo。你也可以选择其他模型。
   
这里的提示对应我们给chatgpt的问题，函数给出的输出则对应chatpgt给我们的答案。

### 2.1 计算1+1

我们来一个简单的例子 - 分别用中英文问问模型

- 中文提示(Prompt in Chinese)： `1+1是什么？`
- 英文提示(Prompt in English)： `What is 1+1?`

In [2]:
# 中文
OpenAiChat().get_completion("1+1是什么？")

'1+1等于2。'

### 2.2 用美式英语表达海盗邮件

上面的简单例子，模型`gpt-3.5-turbo`对我们的关于1+1是什么的提问给出了回答。

现在我们来看一个复杂一点的例子： 

假设我们是电商公司员工，顾客A在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是顾客A给我们的客服中心写来以下邮件：`customer_email`

In [3]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# 东北话
customer_email_zh = """
哎呀妈呀，我都气坏了，我那搅拌机盖子飞出去，把厨房墙上弄得全是奶昔！\
更糟糕的是，这保修还不包清理厨房的费用。哥们儿，我现在真得需要你帮忙啊！
"""

我们的客服人员对于方言措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用普通话对邮件进行翻译以便客服人员可以更好理解。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

In [4]:
# 定义文本表达风
style = "正式的普通话和冷静且尊重的语气"

prompt = f"""
把由三个反引号分隔的文本翻译成一种{style}的风格。
文本:
```{customer_email_zh}```
"""

print(prompt)


把由三个反引号分隔的文本翻译成一种正式的普通话和冷静且尊重的语气的风格。
文本:
```
哎呀妈呀，我都气坏了，我那搅拌机盖子飞出去，把厨房墙上弄得全是奶昔！更糟糕的是，这保修还不包清理厨房的费用。哥们儿，我现在真得需要你帮忙啊！
```



In [5]:
response = OpenAiChat().get_completion(prompt)
response

'尊敬的先生/女士，\n\n我感到非常不安，因为我的搅拌机盖子飞出去了，导致厨房墙壁上都是奶昔。更糟糕的是，保修不包括清理厨房的费用。我真的需要您的帮助，希望您能给予支持。感谢您的理解和合作。\n\n谢谢。'

对比语言风格转换前后，用词更为正式，替换了极端情绪的表达，并表达了感谢。

✨ 你可以尝试修改提示，看可以得到什么不一样的结果😉

## 三、通过LangChain使用OpenAI

在前面一部分，我们通过封装函数`get_completion`直接调用了OpenAI完成了对方言邮件进行了的翻译，得到用平和尊重的语气、正式的普通话表达的邮件。

让我们尝试使用LangChain来实现相同的功能。

### 3.1 模型

从`langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`。 除去OpenAI以外，`langchain.chat_models`还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)。

In [17]:
import os
from langchain_openai import ChatOpenAI

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。

api_key = os.environ.get("CHAT_ANYWHERE_API_KEY")
base_url = "https://api.chatanywhere.com.cn/v1"
chat = ChatOpenAI(temperature=0.0, base_url=base_url, api_key=api_key)
print(type(chat))
chat

<class 'langchain_openai.chat_models.base.ChatOpenAI'>


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1357863e0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x13578f310>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatanywhere.com.cn/v1', openai_proxy='')

### 3.2 提示模版

#### 3.2.1 使用LangChain提示模版


何使用`langchain`来构造提示。

1. 构造提示模版字符串：`template_string`
2. 构造LangChain提示模版
调用`ChatPromptTemplate.from_template()`函数
=> 得到提示模版`prompt_template`
3. 使用模版得到客户消息提示
langchain提示模版`prompt_template`需要两个输入变量： `style` 和 `text`。 这里分别对应
`customer_style`: 我们想要的顾客邮件风格
`customer_email_zh`: 顾客的原始邮件文本。
=> 得到`customer_messages`变量类型为列表(`list`)，每个元素变量类型为(`langchain.schema.HumanMessage`)。

4. 调用chat模型转换客户消息风格
调用[模型](#model)部分定义的chat模型来实现转换客户消息风格。

In [20]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate


customer_email_zh = """
哎呀妈呀，我都气坏了，我那搅拌机盖子飞出去，把厨房墙上弄得全是奶昔！\
更糟糕的是，这保修还不包清理厨房的费用。哥们儿，我现在真得需要你帮忙啊！
"""
customer_style = "正式的普通话和冷静且尊重的语气"
template_string = """
把由三个反引号分隔的文本翻译成这种风格：{style}。
文本:
```{text}```
"""

# 构造LangChain提示模版
prompt_template: ChatPromptTemplate = ChatPromptTemplate.from_template(template_string)
print(type(prompt_template))
print(type(prompt_template.messages))
print(type(prompt_template.messages[0].prompt))
print(prompt_template.messages[0].prompt)

# 使用模版得到客户消息提示
customer_messages: HumanMessage = prompt_template.format_messages(style=style,
                                                    text=customer_email_zh)
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

# 调用chat模型转换客户消息风格
customer_response: AIMessage = chat.__call__(customer_messages)
print(type(customer_response))
print(customer_response)
customer_response.content

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
<class 'list'>
<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['style', 'text'] template='\n把由三个反引号分隔的文本翻译成这种风格：{style}。\n文本:\n```{text}```\n'
<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content='\n把由三个反引号分隔的文本翻译成这种风格：正式的普通话和冷静且尊重的语气。\n文本:\n```\n哎呀妈呀，我都气坏了，我那搅拌机盖子飞出去，把厨房墙上弄得全是奶昔！更糟糕的是，这保修还不包清理厨房的费用。哥们儿，我现在真得需要你帮忙啊！\n```\n'
<class 'langchain_core.messages.ai.AIMessage'>
content='尊敬的先生/女士，我感到非常不安，因为我的搅拌机盖子飞出去了，导致厨房墙壁上都是奶昔。更糟糕的是，保修不包括清理厨房的费用。尊敬的先生/女士，我真的需要您的帮助。感谢您的理解与支持。' response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 165, 'total_tokens': 282}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5b50143f-4928-46b5-a97f-0b87255e97bf-0' usage_metadata={'input_tokens': 165, 'output_tokens': 117, 'total_tokens': 282}


'尊敬的先生/女士，我感到非常不安，因为我的搅拌机盖子飞出去了，导致厨房墙壁上都是奶昔。更糟糕的是，保修不包括清理厨房的费用。尊敬的先生/女士，我真的需要您的帮助。感谢您的理解与支持。'

使用第2步构造的langchain提示模版，来获得我们回复消息提示。

In [21]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""

service_style_pirate = """\
一个有礼貌的语气 \
使用正式的普通话 \
"""
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)


service_response = chat.__call__(service_messages)
service_response.content


把由三个反引号分隔的文本翻译成这种风格：一个有礼貌的语气 使用正式的普通话 。
文本:
```嘿，顾客， 保修不包括厨房的清洁费用， 因为您在启动搅拌机之前 忘记盖上盖子而误用搅拌机, 这是您的错。 倒霉！ 再见！
```



'尊敬的顾客，很抱歉通知您，根据保修条款，厨房清洁费用不在保修范围内。这是因为在使用搅拌机时，您忘记盖上盖子而导致误用搅拌机，这属于您的责任范畴。希望您能理解。祝您好运，再见。'

#### 3.2.2 为什么需要提示模版

在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。**使用提示模版，可以让我们更为方便地重复使用设计好的提示**。下面给出了一个比较长的提示模版案例。学生们线上学习并提交作业，通过以下的提示来实现对学生的提交的作业的评分。

In [22]:
# 中文版
prompt = """ 你的任务是判断学生的解决方案是正确的还是不正确的

要解决该问题，请执行以下操作：
 - 首先，制定自己的问题解决方案
 - 然后将您的解决方案与学生的解决方案进行比较
 并评估学生的解决方案是否正确。
...
使用下面的格式:

问题:
```
问题文本
```
学生的解决方案:
```
学生的解决方案文本
```
实际解决方案:
```
...
制定解决方案的步骤以及您的解决方案请参见此处
```
学生的解决方案和实际解决方案是否相同 \
只计算：
```
是或者不是
```
学生的成绩
```
正确或者不正确
```

问题:
```
{question}
```
学生的解决方案:
```
{student's solution}
```
实际解决方案:

"""

此外，LangChain还提供了提示模版用于一些常用场景。比如自动摘要、问答、连接到SQL数据库、连接到不同的API. 通过使用LangChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

最后，我们在建立大模型应用时，通常希望模型的输出为给定的格式，比如在输出使用特定的关键词来让输出结构化。 下面为一个使用大模型进行链式思考推理例子，对于问题：*What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?*

通过使用LangChain库函数，输出采用"Thought"（思考）、"Action"（行动）、"Observation"（观察）作为链式思考推理的关键词，让输出结构化。

```
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]
```

在补充材料中，可以查看使用LangChain和OpenAI进行链式思考推理的另一个代码实例。

### 3.3 输出解析器

In [30]:
customer_review = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

#### 3.3.1 如果没有输出解析器

对于给定的评价`customer_review`, 我们希望提取信息，并按以下格式输出：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

分析
`response.content`类型为字符串（`str`），而并非字典(`dict`), 直接使用`get`方法会报错。因此，我们需要输出解释器。

In [31]:
from langchain.prompts import ChatPromptTemplate


review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
print(prompt_template)

api_key = os.environ.get("CHAT_ANYWHERE_API_KEY")
base_url = "https://api.chatanywhere.com.cn/v1"
chat = ChatOpenAI(temperature=0.0, base_url=base_url, api_key=api_key)
response = chat(messages)
print(response.content)

response.content.get("礼物")

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='对于以下文本，请从中提取以下信息：\n\n礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。\n\n交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。\n\n价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。\n\n使用以下键将输出格式化为 JSON：\n礼物\n交货天数\n价钱\n\n文本: {text}\n'))]
{
  "礼物": "是的",
  "交货天数": 2,
  "价钱": ["它比其他吹叶机稍微贵一点"]
}


AttributeError: 'str' object has no attribute 'get'

#### 3.3.2 LangChain输出解析器

In [32]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


# 中文
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？"
                                         "如果是，则回答 是的；如果否或未知，则回答 不是。")
delivery_days_schema = ResponseSchema(name="交货天数",
                                      description="产品需要多少天才能到达？如果没有找到该信息，则输出-1。")
price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，"
                                                "并将它们输出为逗号分隔的 Python 列表")
# 构造输出解析器
response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# print(format_instructions)

# 构造LangChain提示模版
prompt_template: ChatPromptTemplate = ChatPromptTemplate.from_template(review_template_2)
messages: HumanMessage = prompt_template.format_messages(text=customer_review,
                                                         format_instructions=format_instructions)
print(messages[0].content)

# 调用chat模型
response: AIMessage = chat.__call__(messages)

# 使用输出解析器解析输出
output_dict = output_parser.parse(response.content)
print(type(output_dict))
output_dict

对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？如果是，则回答 是的；如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表
}
```

<class 'dict'>


{'礼物': '是的', '交货天数': '两天后', '价钱': '它比其他吹叶机稍微贵一点'}

## 四、补充材料

### 4.1 链式思考推理(ReAct)
参考资料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

In [68]:
!pip install -q wikipedia

In [69]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = OpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0,
)

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)


question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)



> Entering new  chain...
Thought: I need to search David Chanoff and find the U.S. Navy admiral he collaborated with. Then I need to find the president under whom the admiral served as the ambassador to the United Kingdom.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought:David Chanoff has collaborated with several individuals, including Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good, and Felix Zandman. I need to search each 

'Bill Clinton'